## Proof of concept for accessing data via tfl APIs and calculating the distance between the user and bike-docking stations in python.

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

### Accessing air quality APIs

In [2]:
# Get all bike points from TfL API
app_id = "f49aabef"
app_key = "740a84c9b887a6ae4de325a3d32ac795"
params = {'app_id': app_id, 'app_key': app_key}
r = requests.get('https://api.tfl.gov.uk/AirQuality', params=params)
air_quality = r.json()

In [3]:
# Today's forecast
air_quality['currentForecast'][0]

{'$id': '2',
 '$type': 'Tfl.Api.Presentation.Entities.CurrentForecast, Tfl.Api.Presentation.Entities',
 'forecastBand': 'Moderate',
 'forecastID': '12421',
 'forecastSummary': 'Moderate ozone air pollution forecast valid from Saturday 17 June to end of Monday 19 June GMT',
 'forecastText': 'Saturday will be dry and sunny for most of the day, feeling warm with periods of cloud cover. A relatively clean air feed from the Atlantic should help dispersion of local emissions and pollution is likely to remain &#39;Low&#39;.&lt;br/&gt;&lt;br/&gt;A fine settle conditions forecast for Sunday and Monday with sunshine for  both days. Very light westerly winds will start to arrive from the east by Sunday evening, importing pollution from the north of the Continent. This air feed, combined with hours of unbroken sunshine will allow ozone production to build through both days, with &#39;Moderate&#39; ozone likely in the afternoons, particularly in suburban and outer-London locations.&lt;br/&gt;&lt;br

In [4]:
# Tomorrow's forecast
air_quality['currentForecast'][1]

{'$id': '3',
 '$type': 'Tfl.Api.Presentation.Entities.CurrentForecast, Tfl.Api.Presentation.Entities',
 'forecastBand': 'Moderate',
 'forecastID': '12421',
 'forecastSummary': 'Moderate ozone air pollution forecast valid from Saturday 17 June to end of Monday 19 June GMT',
 'forecastText': 'Saturday will be dry and sunny for most of the day, feeling warm with periods of cloud cover. A relatively clean air feed from the Atlantic should help dispersion of local emissions and pollution is likely to remain &#39;Low&#39;.&lt;br/&gt;&lt;br/&gt;A fine settle conditions forecast for Sunday and Monday with sunshine for  both days. Very light westerly winds will start to arrive from the east by Sunday evening, importing pollution from the north of the Continent. This air feed, combined with hours of unbroken sunshine will allow ozone production to build through both days, with &#39;Moderate&#39; ozone likely in the afternoons, particularly in suburban and outer-London locations.&lt;br/&gt;&lt;br

### Logic for calculating the distance from the user coordinates to the bikes-docking stations.

In [5]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

#### Testing logic with TicketMaster coordinates

In [6]:
origin = (51.5320521, -0.1092356)
destination = (51.529163, -0.10997)
distance(origin, destination)

0.3252451958196465

### Get bikes-docking stations data point from tfl APIs

In [7]:
r = requests.get('https://api.tfl.gov.uk/BikePoint', params=params)
bike_points = r.json()

In [8]:
res = []
for point in bike_points:
    row = {}
    for e in point['additionalProperties']:
        if e['key'] == 'NbDocks':
            row['docks'] = int(e['value'])
    row['name'] = point['commonName']
    row['id'] = point['id']
    row['lat'] = point['lat']
    row['lon'] = point['lon']
    res.append(row)
res_df = pd.DataFrame(res)

In [9]:
# res

In [10]:
data = {'objects': res}
# with open('TfL_bike_station_data.json', 'wb') as outfile:
#    json.dump(data, outfile)

## Fetching the 3 closest points to the user

In [11]:
distances = []
for row in data['objects']:
    entry = {}
    entry[distance(origin, (row['lat'], row['lon']))] = row['id']
    distances.append(entry)

In [12]:
#distances = sorted(distances)

In [13]:
for e in distances[0:3]:
    for key, value in e.items():
        print(value)

BikePoints_1
BikePoints_2
BikePoints_4


In [14]:
distances[0]

{0.3252451958196465: 'BikePoints_1'}